### Load Variables + Setup

In [15]:
from utils import wholesale_solve, test_primitives, iterative_solve, build_and_eval, is_executable_asp

import os
from llm import LLM

api_key = os.getenv("OPENAI_API_KEY")
llm = LLM(api_key)

directory_path = "instances"

output_folder = "eval_run_iterative"
# Create directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)


5 - maybe
7 
10 
11
12 - maybe
13 - maybe
16
18 - maybe
27 
40
41 - maybe
46
47 - maybe
48 
49 - try
50
54 - if line connect
60 - rotation
64
65
69 - maybe

# Instances to evaluate on:

1, 7, 10, 11, 16, 27, 40, 46, 48, 49, 50, 64, 65

In [2]:
instances_idxs = [1,7,10,11,16,18,27,40,21,46,47,48,49,50,54,60,64,65,69,175]
n_instances = len(instances_idxs)


In [ ]:
# instances_idxs = [1,7]
# n_instances = len(instances_idxs)

In [4]:
n_instances

20

In [5]:

input_primitives = []

for instance in instances_idxs:
    print(f"--- Instance {instance} ---")
    instance_to_run = directory_path+ "/" + "".join([str(instance)])
    print(instance_to_run)

    primitives_text = wholesale_solve(instance=instance_to_run) #finds primitives in examples (except test input)
    input_primitives.append((instance_to_run, primitives_text))
    print(primitives_text)
    print("\n")

--- Instance 1 ---
instances/1
Results for grid_3.lp:
Input grid:
 diagonal_line(input,start(0,1),end(1,2),color(gray),length(2)) diagonal_line(input,start(2,1),end(1,2),color(gray),length(2)) grid_size(input,3,3)

Output grid:
 grid_size(output,3,3) horizontal_line(output,start(0,0),end(0,2),color(yellow),length(3)) horizontal_line(output,start(1,0),end(1,2),color(green),length(3)) horizontal_line(output,start(2,0),end(2,2),color(yellow),length(3))


Results for grid_2.lp:
Input grid:
 diagonal_line(input,start(0,0),end(1,1),color(gray),length(2)) diagonal_line(input,start(2,0),end(1,1),color(gray),length(2)) grid_size(input,3,3)

Output grid:
 grid_size(output,3,3) horizontal_line(output,start(0,0),end(0,2),color(red),length(3)) horizontal_line(output,start(1,0),end(1,2),color(yellow),length(3)) horizontal_line(output,start(2,0),end(2,2),color(red),length(3))


Results for grid_1.lp:
Input grid:
 grid_size(input,3,3) vertical_line(input,start(0,2),end(2,2),color(gray),length(3))

Out

In [6]:
len(input_primitives)

20

In [7]:
n_correct = 0
n_total = n_instances * 3

tracking_accuracy = []

for instance_primitives in input_primitives:

    instance_accuracy = []

    for current_run in range(3):  # Run each instance 3 times
        instance_path = instance_primitives[0]
        primitives = instance_primitives[1]

        total_txt = "STEP 1: Extracted Primitives\n\n"
        print(f"--- Instance {instance_path} ---")

        total_txt += instance_path + "\n" + primitives + "\n"
        print("\n")

        ######## First LLM to predict transformation rules from example primitives ########

        rule_explanation = llm.call("wholesale_pass", primitives=primitives, track_usage=False) #LLM call to predict transformation rules in examples
        total_txt += "STEP 2: Extracted Transformation Rules\n\n" + rule_explanation + "\n"
        print(f'Transformation rules predicted for examples:{rule_explanation}\n')


        ######## Clingo run to retrieve test grid primitives ########

        test_instance_path = os.path.join(instance_path, "grid_test.lp") 
        # print(test_instance_path)
        test_input_primitives = test_primitives(test_instance_path) #finds primitives in test input

        total_txt += "STEP 3: Retrieved Test Input Primitives\n\n" + test_input_primitives + "\n"
        print('-- Primitives found in test input --')
        # print(test_input_primitives)

        ######## Second LLM call to predict test output from test input primitives and predicted rules ########

        test_pass = llm.call("test_pass", rule_explanation=rule_explanation, test_input_primitives=test_input_primitives, track_usage=False)

        total_txt += "STEP 4: Predicted Test Output\n\n" + test_pass + "\n"
        # print(f'Test output predicted:{test_pass}\n')


        ######## Third LLM call to translate final grid prediction to ASP code primitives ########

        output_grid_primitives = llm.call("translate_to_asp", test_input_primitives=test_input_primitives,
                                test_output_prediction=test_pass, 
                                track_usage=False) #LLM call to predict transformation rules in examples

        total_txt += "STEP 5: Translated Test Output to ASP Code Primitives\n\n" + output_grid_primitives + "\n"

        ######## Verify ASP code is executable ASP, re-run if not ########

        outcome = is_executable_asp(output_grid_primitives)

        if outcome == True:
            print("The ASP code is executable and produces output.")
        else:
            print("Not executable:")
            revised_asp = llm.call("fix_asp_code", error_message=outcome, 
                                test_input_primitives=test_input_primitives,
                                test_output_prediction=test_pass,
                                track_usage=False)

            print(f'Revised ASP code:\n{revised_asp}')
            if is_executable_asp(revised_asp) == True:
                print("The revised ASP code is now executable and produces output.")
            else:
                print("The revised ASP code is still not executable.")
                print(f"Here are the attempted primitives:\n{revised_asp}")

            output_grid_primitives = revised_asp
        total_txt += "STEP 6: Verified/Corrected ASP Code Primitives\n\n" + output_grid_primitives + "\n"


        ######## Construct complete output grid from primitives, evaluate correctness ########

        ### Build & evaluate output grid from primitives ###
        # Compare predicted output grid to actual output grid in test input file

        result = build_and_eval(prediction=output_grid_primitives, instance=test_instance_path)

        if result == 'correct_grid':
            print("The predicted output grid matches the actual output grid. Success!")
            n_correct += 1
            instance_accuracy.append(1)
        else:
            print("The predicted output grid does not match the actual output grid. Failure.")
            print(f"Here are the attempted primitives:\n{output_grid_primitives}")
            print(f"Here are the errors in the output:\n{result}")
            instance_accuracy.append(0)

        total_txt += "STEP 7: Evaluation of Predicted Output Grid\n\n" + result + "\n"

        # Write total_txt to file
        instance_name = os.path.basename(instance_path)
        instance_output_dir = os.path.join(output_folder, instance_name)
        os.makedirs(instance_output_dir, exist_ok=True)  # Create instance directory if it doesn't exist
        output_file = os.path.join(instance_output_dir, f"{current_run+1}.txt")
        with open(output_file, 'w') as f:
            f.write(total_txt)
        print(f"Saved results to: {output_file}")

    tracking_accuracy.append(f"{sum(instance_accuracy)}/3")
print(f"Final Score: {n_correct}/{n_total} correct.")
for x in tracking_accuracy:
    print(x)

--- Instance instances/1 ---


Transformation rules predicted for examples:Global rule (applies to all pairs)
- For each row y (0..2) find the gray cell in that row in the input; let its column be x.
- Map column -> color: 0 -> red, 1 -> yellow, 2 -> green.
- Paint the entire row y with that color (i.e., a horizontal line from (y,0) to (y,2)).

Output primitives (general form)
- For each row y: horizontal_line(output, start(y,0), end(y,2), color(C), length(3)) where C is color mapped from the input gray cell column in that row.

Notes
- The grid is 3x3 in every case.
- The grid_1 output (a 3x3 green square) is equivalent to three horizontal green lines (rows 0,1,2 all green) per the same rule.

-- Primitives found in test input --
The ASP code is executable and produces output.
The predicted output grid matches the actual output grid. Success!
Saved results to: eval_run_wholesale/1/1.txt
--- Instance instances/1 ---


Transformation rules predicted for examples:Summary of rule (applies

### Iterative prompting method

In [1]:
from utils import wholesale_solve, test_primitives, iterative_solve, build_and_eval, is_executable_asp

import os
from llm import LLM

api_key = os.getenv("OPENAI_API_KEY")
llm = LLM(api_key)

In [2]:
directory_path = "instances"

output_folder = "eval_run_iterative"
# Create directory if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

In [3]:
# instances_idxs = [1,7]
# n_instances = len(instances_idxs)
# n_instances

In [13]:
# instances_idxs = [1,7,10,11,16,18,27,40,21,46,47,48,49,50,54,60,64,65,69,175]
# instances_idxs = [48,49,50,54,60,64,65,69,175] # Evaluation run interrupted and resumed later
# instances_idxs = [175] # Evaluation run interrupted and resumed later
instances_idxs = [47, 69] # Evaluation run interrupted and resumed later

# Also to run: 47 one time, 69 one time

n_instances = len(instances_idxs)
n_instances

2

In [14]:

input_primitives = []

for instance in instances_idxs:
    print(f"--- Instance {instance} ---")
    instance_to_run = directory_path+ "/" + "".join([str(instance)])
    print(instance_to_run)

    primitives_text = iterative_solve(instance=instance_to_run) #finds primitives in examples (except test input)
    input_primitives.append((instance_to_run, primitives_text))
    print(primitives_text)
    print("\n")

type(primitives_text)

--- Instance 47 ---
instances/47
['Results for grid_2.lp:\nInput grid:\n grid_size(input,5,15) horizontal_line(input,start(4,2),end(4,3),color(red),length(2)) horizontal_line(input,start(4,8),end(4,9),color(red),length(2)) rectangle(input,top_left_corner(2,5),bottom_right_corner(4,6),height(3),width(2),color(yellow)) vertical_line(input,start(2,0),end(4,0),color(yellow),length(3)) vertical_line(input,start(3,1),end(4,1),color(green),length(2)) vertical_line(input,start(3,4),end(4,4),color(green),length(2)) vertical_line(input,start(3,7),end(4,7),color(green),length(2))\n\nOutput grid:\n grid_size(output,5,15) horizontal_line(output,start(4,2),end(4,3),color(red),length(2)) horizontal_line(output,start(4,8),end(4,9),color(red),length(2)) point(output,(4,14),red) rectangle(output,top_left_corner(2,11),bottom_right_corner(4,12),height(3),width(2),color(yellow)) rectangle(output,top_left_corner(2,5),bottom_right_corner(4,6),height(3),width(2),color(yellow)) vertical_line(output,start(2,0),

list

In [11]:
primitives_text[0]

'Results for grid_7.lp:\nInput grid:\n grid_size(input,12,11) isolated_point(input,(1,3),cyan) isolated_point(input,(4,10),cyan) isolated_point(input,(4,6),cyan) isolated_point(input,(5,1),cyan) isolated_point(input,(9,3),cyan)\n\nOutput grid:\n grid_size(output,12,11) horizontal_line(output,start(4,7),end(4,9),color(green),length(3)) isolated_point(output,(5,1),cyan) point(output,(1,3),cyan) point(output,(4,10),cyan) point(output,(4,6),cyan) point(output,(9,3),cyan) vertical_line(output,start(2,3),end(8,3),color(green),length(7))\n\n========================================\n'

In [15]:
n_correct = 0
n_total = n_instances * 3

tracking_accuracy = []

for instance_primitives in input_primitives:

    instance_accuracy = []

    for current_run in range(1):  # Run each instance 3 times
        instance_path = instance_primitives[0]
        primitives_list = instance_primitives[1]

        total_txt = "STEP 1: Extracted Primitives\n\n"
        print(f"--- Instance {instance_path} ---")

        total_txt += instance_path + "\n" + primitives_text[0] + "\n"
        print("\n")

        for idx, model in enumerate(primitives_list):
            print(f'--- Iteration {idx+1} of {len(primitives_list)} ---')
            n_examples = len(primitives_list)
            seen_primitives = str(primitives_list[:idx])
            previous_rule_explanation = rule_explanation if idx > 0 else "N/A"

            if idx == 0: #use first_pass prompt for first example
                rule_explanation = llm.call("first_pass", n_examples=n_examples, primitives=model, track_usage=False) 

            else: #otherwise use iterative_pass prompt to predict rules based on previous examples, rule explanations, and current primitives
                rule_explanation = llm.call("iterative_pass", 
                                            previous_primitives=seen_primitives, 
                                            previous_rule_explanation=previous_rule_explanation, 
                                            idx=idx,
                                            n_examples=n_examples,
                                            primitives=model,
                                            track_usage=False) #LLM call to predict transformation rules in examples
            print(f'Transformation rules predicted for example:{rule_explanation}\n')
            total_txt += f"ITERATION {idx+1} of {n_examples}: Extracted Transformation Rules\n\n" + rule_explanation + "\n"
        total_txt += "FINAL RULE EXPLANATION:\n\n" + rule_explanation + "\n"
        
        ######## Clingo run to retrieve test grid primitives ########

        test_instance_path = os.path.join(instance_path, "grid_test.lp") 
        # print(test_instance_path)
        test_input_primitives = test_primitives(test_instance_path) #finds primitives in test input

        total_txt += "STEP 3: Retrieved Test Input Primitives\n\n" + test_input_primitives + "\n"
        print('-- Primitives found in test input --')
        # print(test_input_primitives)

        ######## Second LLM call to predict test output from test input primitives and predicted rules ########

        test_pass = llm.call("test_pass", rule_explanation=rule_explanation, test_input_primitives=test_input_primitives, track_usage=False)

        total_txt += "STEP 4: Predicted Test Output\n\n" + test_pass + "\n"
        # print(f'Test output predicted:{test_pass}\n')


        ######## Third LLM call to translate final grid prediction to ASP code primitives ########

        output_grid_primitives = llm.call("translate_to_asp", test_input_primitives=test_input_primitives,
                                test_output_prediction=test_pass, 
                                track_usage=False) #LLM call to predict transformation rules in examples

        total_txt += "STEP 5: Translated Test Output to ASP Code Primitives\n\n" + output_grid_primitives + "\n"

        ######## Verify ASP code is executable ASP, re-run if not ########

        outcome = is_executable_asp(output_grid_primitives)

        if outcome == True:
            print("The ASP code is executable and produces output.")
        else:
            print("Not executable:")
            revised_asp = llm.call("fix_asp_code", error_message=outcome, 
                                test_input_primitives=test_input_primitives,
                                test_output_prediction=test_pass,
                                track_usage=False)

            print(f'Revised ASP code:\n{revised_asp}')
            if is_executable_asp(revised_asp) == True:
                print("The revised ASP code is now executable and produces output.")
            else:
                print("The revised ASP code is still not executable.")
                print(f"Here are the attempted primitives:\n{revised_asp}")

            output_grid_primitives = revised_asp
        total_txt += "STEP 6: Verified/Corrected ASP Code Primitives\n\n" + output_grid_primitives + "\n"


        ######## Construct complete output grid from primitives, evaluate correctness ########

        ### Build & evaluate output grid from primitives ###
        # Compare predicted output grid to actual output grid in test input file

        result = build_and_eval(prediction=output_grid_primitives, instance=test_instance_path)

        if result == 'correct_grid':
            print("The predicted output grid matches the actual output grid. Success!")
            n_correct += 1
            instance_accuracy.append(1)
        else:
            print("The predicted output grid does not match the actual output grid. Failure.")
            print(f"Here are the attempted primitives:\n{output_grid_primitives}")
            print(f"Here are the errors in the output:\n{result}")
            instance_accuracy.append(0)

        total_txt += "STEP 7: Evaluation of Predicted Output Grid\n\n" + result + "\n"

        # Write total_txt to file
        instance_name = os.path.basename(instance_path)
        instance_output_dir = os.path.join(output_folder, instance_name)
        os.makedirs(instance_output_dir, exist_ok=True)  # Create instance directory if it doesn't exist
        output_file = os.path.join(instance_output_dir, f"re-run.txt")
        with open(output_file, 'w') as f:
            f.write(total_txt)
        print(f"Saved results to: {output_file}")

    tracking_accuracy.append(f"{sum(instance_accuracy)}/3")
print(f"Final Score: {n_correct}/{n_total} correct.")
for x in tracking_accuracy:
    print(x)

--- Instance instances/47 ---


--- Iteration 1 of 3 ---
Transformation rules predicted for example:Observations / rules
- Grid unchanged (5x15).
- There are three independent repeating motifs across the row dimension:
  - Green vertical 2-cell lines repeat every 3 columns (period = 3). Input had verticals at cols 1,4,7 → output continues the arithmetic sequence to cols 10 and 13 as well.
  - Red horizontal 2-cell segments repeat every 6 columns (period = 6). Input had segments starting at cols 2 and 8 → pattern continues at col 14, but the full length-2 segment would run off-grid, so it is truncated to a single red point at (4,14).
  - Yellow 3x2 rectangles repeat every 6 columns (period = 6). Input had a rectangle with top-left at col 5 → a second identical rectangle appears with top-left at col 11.
- All original shapes are preserved; copies are translations to the right according to the motif periods. If a translated copy would extend beyond grid bounds it is truncated to the visib

In [16]:
import os

count = 0
base_folder = "eval_run_iterative"  # or whatever your base folder is

for root, dirs, files in os.walk(base_folder):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                lines = f.readlines()
                if lines:
                    last_line = lines[-1].strip()
                    if last_line == 'correct_grid':
                        count += 1
                        print(f"Found 'correct_grid' in {file_path}. Count: {count}")
                    else:
                        print(f"Last line is '{last_line}', not 'correct_grid'")
                else:
                    print(f"File {file_path} is empty")

print(f"Total files with 'correct_grid' as last line: {count}")
print(f"Accuracy: {count}/60. = {count/60:.2%}")

Found 'correct_grid' in eval_run_iterative/50/3.txt. Count: 1
Last line is 'incorrect_cell((0,8),(predicted(black),correct(blue))) incorrect_cell((1,7),(predicted(black),correct(blue))) incorrect_cell((1,8),(predicted(black),correct(blue))) incorrect_cell((1,9),(predicted(black),correct(blue))) incorrect_cell((2,2),(predicted(black),correct(red))) incorrect_cell((2,8),(predicted(black),correct(blue))) incorrect_cell((3,1),(predicted(black),correct(red))) incorrect_cell((3,2),(predicted(black),correct(red))) incorrect_cell((3,3),(predicted(black),correct(red))) incorrect_cell((4,2),(predicted(black),correct(red))) incorrect_cell((6,6),(predicted(black),correct(blue))) incorrect_cell((6,7),(predicted(black),correct(blue))) incorrect_cell((7,2),(predicted(black),correct(red))) incorrect_cell((8,1),(predicted(black),correct(red))) incorrect_cell((8,2),(predicted(black),correct(red))) incorrect_cell((8,3),(predicted(black),correct(red))) incorrect_cell((9,2),(predicted(black),correct(red)))

### Results breakdown

Wholesale:
Final Score: 44/60 correct.
3/3 - 1
3/3 - 7
1/3 - 10 
1/3 - 11
3/3 - 16
0/3 - 18
3/3 - 21
0/3 - 27
0/3 - 40
3/3 - 46
3/3 - 47
3/3 - 48
3/3 - 49
1/3 - 50
2/3 - 54
3/3 - 60
3/3 - 64
3/3 - 65
3/3 - 69
3/3 - 175

Iterative:

1/3 - 1
1/3 - 7
3/3 - 10
1/3 - 11
3/3 - 16
0/3 - 18
0/3 - 21
3/3 - 27
0/3 - 40
3/3 - 46
0/3 - 47
2/3 - 48
1/3 - 49
2/3 - 50
3/3 - 54
3/3 - 60
3/3 - 64
3/3 - 65
2/3 - 69
3/3 - 175

In [ ]:
import os

base_folder = "eval_run_iterative"  
subdirectory_results = {}

for root, dirs, files in os.walk(base_folder):
    # Skip the base folder itself
    if root == base_folder:
        continue
    
    subdirectory_name = os.path.basename(root)
    correct_count = 0
    total_files = 0
    
    for file in files:
        if file.endswith('.txt'):
            total_files += 1
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                lines = f.readlines()
                if lines:
                    last_line = lines[-1].strip()
                    if last_line == 'correct_grid':
                        correct_count += 1
    
    if total_files > 0:
        subdirectory_results[subdirectory_name] = (correct_count, total_files)

# Display results in the requested format
for subdirectory, (correct, total) in subdirectory_results.items():
    print(f"{correct}/{total} - {subdirectory}")

# Calculate overall accuracy
total_correct = sum(correct for correct, total in subdirectory_results.values())
total_files = sum(total for correct, total in subdirectory_results.values())
print(f"\nOverall accuracy: {total_correct}/{total_files} = {total_correct/total_files:.2%}")

2/3 - 50
2/3 - 69
3/3 - 60
0/3 - 18
3/3 - 27
1/3 - 11
1/3 - 7
3/3 - 16
1/3 - 1
3/3 - 10
3/3 - 175
0/3 - 21
3/3 - 65
3/3 - 54
3/3 - 64
3/3 - 46
2/3 - 48
1/3 - 49
0/3 - 40
0/3 - 47

Overall accuracy: 37/60 = 61.67%
